In [ ]:
!pip install transformers
!pip install rouge-score
!pip install nltk
!pip install evaluate

from IPython.display import clear_output
clear_output()

In [ ]:
import kagglehub
import pandas as pd
import torch
import transformers
import nltk
import numpy as np

from evaluate import load
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
recipe_path = kagglehub.dataset_download("shuyangli94/food-com-recipes-and-user-interactions")
print("Path to recipe dataset files:", recipe_path)
recipe_raw_df = pd.read_csv(recipe_path + '/RAW_recipes.csv')

KeyboardInterrupt: 

In [ ]:
# recipe_df.head()
recipe_raw_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
model_checkpoint = 't5-small'
nltk.download('punkt')
rouge_metric = load('rouge')
bleu_metric = load('bleu')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
"""
copy_recipe_df = recipe_raw_df.copy()
copy_recipe_df = copy_recipe_df.dropna(subset=["steps"])
copy_recipe_df['name_and_ingredients'] = copy_recipe_df['name'] + ' ' + copy_recipe_df['ingredients']
subset_df = copy_recipe_df.sample(n = 100000, random_state = 42)

# Split is 80%, 10%, 10%
train_val_data, test_data = train_test_split(subset_df, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=1/9, random_state=42)

test_set = test_data[:500] # Only test on 500 to allow comparison.
train_datasets = Dataset.from_pandas(train_data)
val_datasets = Dataset.from_pandas(val_data)
test_datasets = Dataset.from_pandas(test_data)
dataset_dict = DatasetDict({"train": train_datasets, "validation": val_datasets, "test": test_datasets})

dataset_dict.save_to_disk('/content/drive/MyDrive/CSCI544 Project')

subset_df.head()
"""
dataset_dict = load_from_disk('/content/drive/MyDrive/CSCI544 Project')
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients', 'name_and_ingredients', '__index_level_0__'],
        num_rows: 80000
    })
    validation: Dataset({
        features: ['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients', 'name_and_ingredients', '__index_level_0__'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients', 'name_and_ingredients', '__index_level_0__'],
        num_rows: 10000
    })
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 512 # 1024
max_target_length = 128
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["name_and_ingredients"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["steps"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
finetune_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
batch_size = 32# 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=finetune_model)

In [ ]:
"""
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}
"""
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    decoded_preds_rouge = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels_rouge = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE
    rouge_result = rouge_metric.compute(predictions=decoded_preds_rouge, references=decoded_labels_rouge, use_stemmer=True, use_aggregator=True)
    rouge_result = {key: value * 100 for key, value in rouge_result.items()}

    # Compute BLEU
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels, smooth=True)

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    gen_len = np.mean(prediction_lens)

    # Combine results
    result = {**rouge_result, **bleu_result, 'gen_len': round(gen_len, 4)}

    return result

In [ ]:
finetune_trainer = Seq2SeqTrainer(
    finetune_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
finetune_trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length,Gen Len
1,2.198400,2.001368,12.950989,4.478202,10.679113,10.686929,0.000968,"[0.5401622316747453, 0.1615876666546879, 0.07887827219962885, 0.04186927760984799]",0.007426,0.169412,121307,716048,18.998400
2,2.166200,1.979475,12.877383,4.518633,10.672577,10.677434,0.000966,"[0.5362214736285875, 0.1626498434068901, 0.07995767237628071, 0.04261600597041135]",0.007357,0.169144,121115,716048,18.998400


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "[0.5401622316747453, 0.1615876666546879, 0.07887827219962885, 0.04186927760984799]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "[0.5362214736285875, 0.1626498434068901, 0.07995767237628071, 0.04261600597041135]" of type <class 'list'> for k

TrainOutput(global_step=5000, training_loss=2.206025537109375, metrics={'train_runtime': 1734.6099, 'train_samples_per_second': 92.24, 'train_steps_per_second': 2.882, 'total_flos': 6250211274915840.0, 'train_loss': 2.206025537109375, 'epoch': 2.0})

In [ ]:
# Now evaluate on the test set
test_predictions = finetune_trainer.predict(tokenized_datasets["test"])
print(test_predictions.metrics)

{'test_loss': 1.9857748746871948, 'test_rouge1': 12.896115059364577, 'test_rouge2': 4.513192580922481, 'test_rougeL': 10.649730782735949, 'test_rougeLsum': 10.661987754419775, 'test_bleu': 0.0009672651868465193, 'test_precisions': [0.5344464711659885, 0.1620921382883977, 0.07992368449668344, 0.04280433089437137], 'test_brevity_penalty': 0.007372049006026131, 'test_length_ratio': 0.16920303162763548, 'test_translation_length': 121158, 'test_reference_length': 716051, 'test_gen_len': 18.9995, 'test_runtime': 165.568, 'test_samples_per_second': 60.398, 'test_steps_per_second': 1.89}


In [ ]:
import math

precisions = test_predictions.metrics['test_precisions']
bp = test_predictions.metrics['test_brevity_penalty']

for i in range(len(precisions)):
  p_log_sum = sum((1. / (i + 1)) * math.log(p) for p in precisions[: i+1])
  geo_mean = math.exp(p_log_sum)

  print("BLEU " + str(i + 1) + ": ", geo_mean * bp)

BLEU 1:  0.003939965576533399
BLEU 2:  0.0021698079505419457
BLEU 3:  0.0014050833933398285
BLEU 4:  0.0009672651868465193


In [ ]:
save_path = '/content/drive/MyDrive/CSCI544 Project/T5 Runs/10_28_2024'
finetune_model.save_pretrained(save_path)

In [ ]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=base_model)

base_trainer = Seq2SeqTrainer(
    base_model,
    args,
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

base_trainer.evaluate()

base_test_predictions = base_trainer.predict(tokenized_datasets["test"])
print(base_test_predictions.metrics)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'test_loss': 4.876565456390381, 'test_model_preparation_time': 0.006, 'test_rouge1': 7.969906930062025, 'test_rouge2': 1.632924731615263, 'test_rougeL': 6.743149548668853, 'test_rougeLsum': 6.755310101194423, 'test_bleu': 0.0004, 'test_gen_len': 18.9974, 'test_runtime': 160.1833, 'test_samples_per_second': 62.428, 'test_steps_per_second': 1.954}


In [ ]:
task_prefix = ""
tasks = ["Recipe Name: Lemon Garlic Chicken. Ingredients:chicken breast, garlic, olive oil, lemon, thyme",
         "Recipe Name: Classic Vanilla Cake. Ingredients:flour, sugar, eggs, butter, vanilla extract"]

finetune_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/CSCI544 Project/T5 Runs/10_28_2024")
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
input_ids = tokenizer(tasks[0], return_tensors="pt").input_ids
outputs = base_model.generate(input_ids, max_new_tokens = 128)
finetune_outputs = finetune_model.generate(input_ids, max_new_tokens = 128)

print("Input: ", tasks[0])
print("Base model: ", tokenizer.batch_decode(outputs, skip_special_tokens=True))
print("Finetuned: ", tokenizer.batch_decode(finetune_outputs, skip_special_tokens=True))

Input:  Recipe Name: Lemon Garlic Chicken. Ingredients:chicken breast, garlic, olive oil, lemon, thyme
Base model:  ['garlic, olive oil, lemon, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme, thyme,']
Finetuned:  ['Ingredients:cook chicken breast, garlic, olive oil, lemon, thyme and garlic in a large bowl.']


In [ ]:
input_ids = tokenizer(tasks[0], return_tensors="pt").input_ids
for i in range(5):
  finetune_outputs = finetune_model.generate(input_ids, max_new_tokens = 128)
  print("Timestep ", i + 1, ": ", tokenizer.batch_decode(finetune_outputs, skip_special_tokens=True)[0])
  input_ids = finetune_outputs

Timestep  1 :  Ingredients:cook chicken breast, garlic, olive oil, lemon, thyme, thyme, and garlic in a large bowl.
Timestep  2 :  :cook chicken breast, garlic, olive oil, lemon, thyme, thyme, and garlic in a large bowl.
Timestep  3 :  cook chicken breast, garlic, olive oil, lemon, thyme, and garlic in a large bowl.
Timestep  4 :  a large bowl ofa large bowl over medium-high heat. In a large bowl, over medium-high heat, chicken breast, garlic, olive oil, and lemon juice.
Timestep  5 :   chicken breast and in  combine chicken breast, chicken breast, garlic, and lemon juice. In  combine chicken breast, chicken breast, garlic, and olive oil. In a large bowl, combine chicken breast, chicken breast, and lemon juice. In a large bowl, combine chicken breast, chicken breast, and garlic.


In [ ]:
bleu = load("bleu")

# Example references and predictions
references = [["the cat is on the mat"], ["there is a cat on the mat"]]
predictions = ["the cat is on the mat", "a cat is on the mat"]

# Compute BLEU score
results = bleu.compute(predictions=predictions, references=references, max_order = 3)

print(results)
print(f"BLEU score: {results['bleu']}")

{'bleu': 0.7302397358200271, 'precisions': [1.0, 0.8, 0.625], 'brevity_penalty': 0.9200444146293233, 'length_ratio': 0.9230769230769231, 'translation_length': 12, 'reference_length': 13}
BLEU score: 0.7302397358200271
